# 데이터셋 출처
https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database https://ko.wikipedia.org/wiki/%EA%B2%B0%EC%A0%95_%ED%8A%B8%EB%A6%AC_%ED%95%99%EC%8A%B5%EB%B2%95
- Pregnancies : 임신 횟수
- Glucose : 2시간 동안의 경구 포도당 내성 검사에서 혈장 포도당 농도
- BloodPressure : 이완기 혈압 (mm Hg)
- SkinThickness : 삼두근 피부 주름 두께 (mm), 체지방을 추정하는데 사용되는 값
- Insulin : 2시간 혈청 인슐린 (mu U / ml)
- BMI : 체질량 지수 (체중kg / 키(m)^2)
- DiabetesPedigreeFunction : 당뇨병 혈통 기능
- Age : 나이
- Outcome : 768개 중에 268개의 결과 클래스 변수(0 또는 1)는 1이고 나머지는 0입니다.

# 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# 데이터셋 로드

In [2]:
# 전처리한 데이터
df = pd.read_csv("diabetes_feature.csv")
df.shape

(768, 16)

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Pregnancies_high,Age_low,Age_middle,Age_high,Insulin_nan,Insulin_log,low_glu_insulin
0,6,148,72,35,0,33.6,0.627,50,1,False,False,True,False,169.5,5.138735,False
1,1,85,66,29,0,26.6,0.351,31,0,False,False,True,False,102.5,4.639572,True
2,8,183,64,0,0,23.3,0.672,32,1,True,False,True,False,169.5,5.138735,False
3,1,89,66,23,94,28.1,0.167,21,0,False,True,False,False,94.0,4.553877,True
4,0,137,40,35,168,43.1,2.288,33,1,False,False,True,False,168.0,5.129899,False


# 학습과 예측에 사용할 데이터셋 만들기

In [4]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome', 'Pregnancies_high',
       'Age_low', 'Age_middle', 'Age_high', 'Insulin_nan', 'Insulin_log',
       'low_glu_insulin'],
      dtype='object')

In [5]:
X = df[['Glucose', 'BloodPressure', 'SkinThickness', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Pregnancies_high',
       'Insulin_nan', 'low_glu_insulin']]
X.shape

(768, 9)

In [6]:
y = df["Outcome"]
y.shape

(768,)

In [8]:
# 사이킷런에서 제공하는 train_test_split을 사용하여 데이터 나누기
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 9), (154, 9), (614,), (154,))

# 여러 개의 알고리즘을 사용하여 비교하기

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

estimators = [DecisionTreeClassifier(random_state=42), 
             RandomForestClassifier(random_state=42), 
             GradientBoostingClassifier(random_state=42)
             ] 
estimators

[DecisionTreeClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 GradientBoostingClassifier(random_state=42)]

In [20]:
# 트리의 최대 깊이 지정
max_depth = np.random.randint(2, 20, 10)
max_depth

array([ 3, 19, 12,  3,  5,  6, 19, 11,  5, 10])

In [17]:
# 각 트리별로 몇 개의 컬럼을 선택할 것인지, 각 트리의 랜덤성 결정, 클수록 과적합을 줄인다
max_features = np.random.uniform(0.3, 1.0, 10)
max_features

array([0.55499246, 0.82625823, 0.52046179, 0.60755233, 0.3240168 ,
       0.59676435, 0.89764169, 0.3383786 , 0.81114311, 0.97923933])

In [36]:
# 반복문을 통해 results 리스트에 각각의 모델 이름 담기
results = []
for estimator in estimators:
    result = []
    result.append(estimator.__class__.__name__)
    results.append(result)
results

[['DecisionTreeClassifier'],
 ['RandomForestClassifier'],
 ['GradientBoostingClassifier']]

In [51]:
from sklearn. model_selection import RandomizedSearchCV

max_depth = np.random.randint(2, 20, 10)
max_features = np.random.uniform(0.5, 1.0, 10)

param_distributions = {"max_depth":max_depth, "max_features":max_features}

results = []
for estimator in estimators:
    result = []
    if estimator.__class__.__name__ != "DecisionTreeClassifier":
        param_distributions["n_estimators"] = np.random.randint(100, 200, 10)
    clf = RandomizedSearchCV(estimator, 
                       param_distributions, 
                       n_iter=100, 
                       scoring="accuracy", 
                       n_jobs=-1,
                       cv=5,
                       verbose=2)

    clf.fit(X_train, y_train)
    result.append(estimator.__class__.__name__)
    result.append(clf.best_params_)
    result.append(clf.best_score_)
    result.append(clf.score(X_test,y_test))
    result.append(clf.cv_results_)
    results.append(result)
    
    
# param_distributions에 랜덤 서치를 위한 변수들을 딕셔너리 형태로 넣기
# n_iter으로 100번을 반복 학습
# n_jobs를 -1으로 지정하여 사용할 수 있는 자원을 모두 활용
# cv를 5로 하여 fold를 5개로 나누기 -> 많이 나누면 나눌수록 좋은 점수를 받을 수 있다
# verbose를 2로 지정하여 로그 찍기
# 함수 수행시 발생하는 상세한 정보들을 표준 출력으로 자세히 내보낼 것인가?의 의미
# verbose: 0: 출력 X / 1: 자세히 / 2: 함축적인 정보만 출력
# fold를 5개로 나누는데(cv), 10번을 돌면서 학습(n_iter) -> 총 50번을 학습한다.
# n_iter를 올리면 더 좋은 점수가 나올 것이다. 다만 시간이 걸릴 수도..
# test 데이터의 정답도 있기 때문에 result에 비교를 위해 추가해준다.
# 만약 결정트리가 아니라면 n_estimators 파라미터를 추가해주는 조건문을 넣었다.(다른 모델에는 있기 때문에 성능을 올리기 위하여)
# n_estimators: 트리의 개수 -> 크기가 클수록 모델 학습에 시간이 오래걸린다

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [50]:
# 모델에 각각 접근하여 상위에 있는 모델 순서대로 보거나 정확한 parameter 설정 수치를 볼 수 있다
# 좋은 성능이 나오는 구간으로 계속 iteration을 돌릴 필요가 있다
# 하이퍼 파라미터 튜닝을 여러 번 할수록 더 좋은 성능을 얻을 수 있다
pd.DataFrame(results, columns=["estimator", "best_params", "train_score", "test_score", "cv_result"])

,estimator,best_params,train_score,test_score,cv_result
0,DecisionTreeClassifier,"{'max_features': 0.8641345412692931, 'max_dept...",0.868106,0.876623,"{'mean_fit_time': [0.012396860122680663, 0.015..."
1,RandomForestClassifier,"{'n_estimators': 114, 'max_features': 0.773008...",0.903972,0.857143,"{'mean_fit_time': [1.215045166015625, 1.470911..."
2,GradientBoostingClassifier,"{'n_estimators': 180, 'max_features': 0.583157...",0.903945,0.870130,"{'mean_fit_time': [3.559855079650879, 1.454999..."


In [24]:
# 여러 알고리즘을 반복문을 통해 가장 좋은 파라미터와 점수를 나오게 돌려놓았으므로 아래 코드들은 필요 없다
# 가장 좋은 하이퍼 파라미터 찾기
clf.best_params_


# max_features가 0.88일 때 좋은 성능을 낸 다는 말
# max_depth가 4일 때 좋은 성능을 낸다는 말

{'max_features': 0.8804441731197319, 'max_depth': 4}

In [25]:
clf.best_score_

0.8681060908969747